In [12]:
import http.client, urllib, base64, requests
import cognitive_face as CF 
from cognitive_face import util
import time

time: 15.1 ms


In [13]:
%install_ext https://raw.github.com/cpcloud/ipython-autotime/master/autotime.py
%load_ext autotime

/Users/Amiros/anaconda/lib/python3.5/site-packages/IPython/core/magics/extension.py:47: UserWarning: %install_ext` is deprecated, please distribute your extension as a python package.
  "as a python package.", UserWarning)


Installed autotime.py. To use it, type:
  %load_ext autotime
The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 392 ms


In [14]:
#Emotion Analysis parameters
_url = 'https://api.projectoxford.ai/emotion/v1.0/recognize'
_key = '183aea69820e4dfda0176db816fc1f72' #Here you have to paste your primary key
_maxNumRetries = 5

#Face detection parametrs
CF.Key.set('d78211e12bba41a991382ade5d68745b') #set the key

time: 4.17 ms


In [15]:
# Image path
urlImage= ''

# Emtional Anlaysis function
def processRequest( json, data, headers, params ):

    retries = 0
    result = None

    while True:

        response = requests.request( 'post', _url, json = json, data = data, headers = headers, params = params )

        if response.status_code == 429: 

            print( "Message: %s" % ( response.json()['error']['message'] ) )

            if retries <= _maxNumRetries: 
                time.sleep(1) 
                retries += 1
                continue
            else: 
                print( 'Error: failed after retrying!' )
                break

        elif response.status_code == 200 or response.status_code == 201:

            if 'content-length' in response.headers and int(response.headers['content-length']) == 0: 
                result = None 
            elif 'content-type' in response.headers and isinstance(response.headers['content-type'], str): 
                if 'application/json' in response.headers['content-type'].lower(): 
                    result = response.json() if response.content else None 
                elif 'image' in response.headers['content-type'].lower(): 
                    result = response.content
        else:
            print( "Error code: %d" % ( response.status_code ) )
            print( "Message: %s" % ( response.json()['error']['message'] ) )

        break
        
    return result


headers = dict()
headers['Ocp-Apim-Subscription-Key'] = _key
headers['Content-Type'] = 'application/json' 

json = { 'url': urlImage } 
data = None
params = None

time: 73.1 ms


In [16]:
# Face detect function
def detect(image, face_id=True, landmarks=False, faceRectangle= False, attributes='gender'):
    
    url = 'detect'
    headers, data, json = util.parse_image(image)
    params = {
        'returnFaceId': face_id and 'true',
        'returnFaceLandmarks': landmarks and 'false',
        'returnFaceAttributes': attributes,
    }

    return util.request('POST', url, headers=headers, params=params, json=json,
                        data=data)

time: 5.75 ms


In [17]:
#Get all the frames from S3
#start with 355
url_list = []
for i in range(355,5920,5):
    url = "https://s3-us-west-2.amazonaws.com/debateinemotion/debate1/frame%d.jpg" % i
    url_list.append(url)

time: 5.06 ms


In [18]:
# Face Detection & Emotioin Analysis 
emotions = {}
count = 355
#355

for single_image in url_list:

    json = { 'url': single_image } 
    emotions[count] = detect(single_image)
    processed = processRequest(json, data, headers, params)
    
    try:
        emotions[count].append(processed[0])  
    except:
        pass 

    count += 5
    
    time.sleep(3)

status_code: 400
response: {"error":{"code":"InvalidURL","message":"Invalid image URL or error downloading from target server. Remote server error returned: \"Response status code does not indicate success: 403 (Forbidden).\""}}


CognitiveFaceException: Error when calling Cognitive Face API:
	status_code: 400
	code: InvalidURL
	message: Invalid image URL or error downloading from target server. Remote server error returned: "Response status code does not indicate success: 403 (Forbidden)."


time: 7min 55s


In [7]:
# Face Detection & Emotion Analysis - UPDATED
processed = {}
face_detection = {}
count = 355
#355

for single_image in url_list:
 
    face_detection[count] = detect(single_image)
    
    json = { 'url': single_image } 
    processed[count] = processRequest(json, data, headers, params)    
    
    try:
        if face_detection[count][0]['faceRectangle'] == processed[count][0]['faceRectangle']:
            face_detection[count][0].update(processed[count][0])
            face_detection[count][1].update(processed[count][1])
        if  face_detection[count][0]['faceRectangle'] == processed[count][1]['faceRectangle']:
            face_detection[count][0].update(processed[count][1])
            face_detection[count][1].update(processed[count][0])
    except:
        pass
    
    count += 5
    
    time.sleep(3)

time: 1h 40min 26s


In [9]:
face_detection

{2050: [{'faceAttributes': {'gender': 'male'},
   'faceId': '6fa54e8f-3c62-4f00-a221-3453da1e3815',
   'faceRectangle': {'height': 187, 'left': 236, 'top': 192, 'width': 187},
   'scores': {'anger': 0.494146138,
    'contempt': 0.0153631493,
    'disgust': 0.00263129943,
    'fear': 5.64368565e-05,
    'happiness': 1.39012191e-05,
    'neutral': 0.4417074,
    'sadness': 0.04594154,
    'surprise': 0.0001401298}},
  {'faceAttributes': {'gender': 'female'},
   'faceId': '58b56d54-c2f0-49c0-8d35-15c0e534076d',
   'faceRectangle': {'height': 178, 'left': 842, 'top': 158, 'width': 178},
   'scores': {'anger': 8.032862e-05,
    'contempt': 0.002508147,
    'disgust': 0.000283529022,
    'fear': 6.545869e-06,
    'happiness': 0.7335016,
    'neutral': 0.25900358,
    'sadness': 0.000250928919,
    'surprise': 0.004365326}}],
 4100: [{'faceAttributes': {'gender': 'male'},
   'faceId': 'af1a170f-3002-4e06-a4e1-79cf254b1090',
   'faceRectangle': {'height': 189, 'left': 251, 'top': 182, 'width':

time: 1.09 s


In [11]:
# Save the result as Json
import json
with open('debate1_full2.json', 'w') as fp:
    json.dump(face_detection, fp, sort_keys=True)

time: 215 ms
